### 1. Imports + Path Setup

In [1]:
import sys
from pathlib import Path

# notebooks/ -> project root
ROOT = Path.cwd().parents[0]
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("Project root:", ROOT)


Project root: C:\Users\hyeon\Documents\miniconda_medimg_env\medimg_baseline_cls


### 2. Config + reproducibility

In [3]:
data_dir = r"C:\Users\hyeon\Documents\miniconda_medimg_env\data\chest_xray"

In [4]:
from src.config import Config, seed_everything, ensure_dirs
from src.utils import env_report, save_json

cfg = Config(
    project_name="medimg_baseline_cls",
    data_root=data_dir,
    output_root="outputs",
    image_size=(224, 224),
    batch_size=32,
    num_workers=0,        # start 0 on Windows; increase to 2 or 4 once stable
    pin_memory=True,
    max_epochs=10,
    head_epochs=2,
    lr_head=3e-3,
    lr_finetune=1e-3,     # consider 3e-4 later for smoother FT
    weight_decay=1e-4,
    rebuild_balanced_val=True,
    val_n_per_class=200,
    use_weighted_sampler=True,
    seed=42,
)

seed_everything(cfg.seed, cfg.deterministic)
ensure_dirs(cfg)

print(cfg)
print("Env:", env_report())


Config(project_name='medimg_baseline_cls', data_root='C:\\Users\\hyeon\\Documents\\miniconda_medimg_env\\data\\chest_xray', output_root='outputs', class_names=('NORMAL', 'PNEUMONIA'), pos_class_name='PNEUMONIA', image_size=(224, 224), rebuild_balanced_val=True, val_n_per_class=200, batch_size=32, num_workers=0, pin_memory=True, max_epochs=10, head_epochs=2, lr_head=0.003, lr_finetune=0.001, weight_decay=0.0001, use_weighted_sampler=True, seed=42, deterministic=False, device='cuda', save_best_by='val_ap', save_history=True, gradcam_alpha=0.35)
Env: {'os': 'Windows-10-10.0.26200-SP0', 'python': '3.11.14', 'machine': 'AMD64', 'processor': 'Intel64 Family 6 Model 183 Stepping 1, GenuineIntel', 'torch_version': '2.5.1', 'cuda_available': True, 'cuda_version': '12.1', 'cudnn_version': 90100, 'gpu_name': 'NVIDIA GeForce RTX 4090', 'gpu_count': 1}


### 3. Data (items, datasets, loaders)

In [5]:
from src.data import build_datasets, build_loaders, label_counts

ds = build_datasets(
    root_dir=cfg.data_root,
    class_names=cfg.class_names,
    image_size=cfg.image_size,
    rebuild_balanced_val=cfg.rebuild_balanced_val,
    val_n_per_class=cfg.val_n_per_class,
    seed=cfg.seed,
)

print("Counts:")
print("train:", label_counts(ds["train_items"], len(cfg.class_names)))
print("val  :", label_counts(ds["val_items"], len(cfg.class_names)))
print("test :", label_counts(ds["test_items"], len(cfg.class_names)))

loaders = build_loaders(
    train_ds=ds["train_ds"],
    val_ds=ds["val_ds"],
    test_ds=ds["test_ds"],
    train_items=ds["train_items"],
    class_names=cfg.class_names,
    batch_size=cfg.batch_size,
    num_workers=cfg.num_workers,
    pin_memory=cfg.pin_memory,
    use_weighted_sampler=cfg.use_weighted_sampler,
)

# Sanity batch
b = next(iter(loaders["train_loader"]))
print("Batch image shape:", b["image"].shape)
print("Batch labels:", {int(k): int((b["label"].numpy()==k).sum()) for k in set(b["label"].numpy().tolist())})


C:\Users\hyeon\miniconda3\envs\medimg\Lib\site-packages\ignite\handlers\checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer
C:\Users\hyeon\miniconda3\envs\medimg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Counts:
train: {0: 1141, 1: 3675}
val  : {0: 200, 1: 200}
test : {0: 234, 1: 390}
Batch image shape: torch.Size([32, 3, 224, 224])
Batch labels: {0: 22, 1: 10}


### 4. Model + freeze backbone

In [6]:
from src.models import build_model, freeze_backbone, get_head_prefixes, get_gradcam_target_layer

arch = "resnet18"  # swap to 'densenet121' later if desired
model = build_model(arch=arch, num_classes=len(cfg.class_names), pretrained=True, device=cfg.device)

# Freeze backbone for head-only stage
freeze_backbone(model, head_prefixes=get_head_prefixes(arch))
print("Trainable tensors:", sum(p.requires_grad for p in model.parameters()), "/", len(list(model.parameters())))

# Save gradcam target layer reference for later
target_layer = get_gradcam_target_layer(model, arch)
print("Grad-CAM target layer:", target_layer)


Trainable tensors: 2 / 62
Grad-CAM target layer: Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)


### 5. Train (head-only → fine-tune) + save run metadata

In [7]:
from src.train import run_training
from src.utils import save_json

result = run_training(
    cfg=cfg,
    model=model,
    train_loader=loaders["train_loader"],
    val_loader=loaders["val_loader"],
    test_loader=loaders["test_loader"],
)

print("Run saved to:", result["run_dir"])
print("Best checkpoint:", result["best_ckpt_path"])
print("VAL summary:", result["val_summary"])
print("TEST summary:", result["test_summary"])

# Save environment report into the run directory for reproducibility
save_json(env_report(), str(Path(result["run_dir"]) / "env_report.json"))

# (Optional) Keep these in memory for the next notebook
RUN_DIR = result["run_dir"]
BEST_CKPT = result["best_ckpt_path"]
print("RUN_DIR =", RUN_DIR)
print("BEST_CKPT =", BEST_CKPT)



=== Stage A: Head-only training ===

[Epoch 1/10] (Head-only)


[Head] tr_loss=0.2757 tr_acc~=0.912 preds={0: 170, 1: 150} | val_loss=0.1876 val_acc=0.920 val_AP=0.987 | test_AP=0.950 | 128.2s

[Epoch 2/10] (Head-only)


[Head] tr_loss=0.1927 tr_acc~=0.953 preds={0: 174, 1: 146} | val_loss=0.1392 val_acc=0.950 val_AP=0.990 | test_AP=0.952 | 128.6s

=== Stage B: Full fine-tuning ===

[Epoch 3/10] (Fine-tuning)


[FT ] tr_loss=0.1318 tr_acc~=0.966 preds={0: 175, 1: 145} | val_loss=0.1051 val_acc=0.975 val_AP=0.998 | test_AP=0.973 | 129.3s

[Epoch 4/10] (Fine-tuning)


[FT ] tr_loss=0.0547 tr_acc~=0.991 preds={0: 175, 1: 145} | val_loss=0.0507 val_acc=0.983 val_AP=0.999 | test_AP=0.974 | 130.5s

[Epoch 5/10] (Fine-tuning)


...

[FT ] tr_loss=0.0200 tr_acc~=0.709 preds={0: 255, 1: 65} | val_loss=1.5010 val_acc=0.708 val_AP=0.999 | test_AP=0.951 | 129.7s

[Epoch 7/10] (Fine-tuning)


[FT ] tr_loss=0.0350 tr_acc~=0.994 preds={0: 164, 1: 156} | val_loss=0.0313 val_acc=0.988 val_AP=0.999 | test_AP=0.948 | 135.2s

[Epoch 8/10] (Fine-tuning)


[FT ] tr_loss=0.0198 tr_acc~=0.997 preds={0: 159, 1: 161} | val_loss=0.0337 val_acc=0.988 val_AP=0.999 | test_AP=0.950 | 128.8s

[Epoch 9/10] (Fine-tuning)


[FT ] tr_loss=0.0121 tr_acc~=0.997 preds={0: 164, 1: 156} | val_loss=0.0409 val_acc=0.993 val_AP=1.000 | test_AP=0.950 | 130.4s

[Epoch 10/10] (Fine-tuning)


[FT ] tr_loss=0.0132 tr_acc~=0.978 preds={0: 165, 1: 155} | val_loss=0.0572 val_acc=0.970 val_AP=0.999 | test_AP=0.815 | 132.1s
Run saved to: outputs\runs\medimg_baseline_cls_20260110_182220
Best checkpoint: outputs\runs\medimg_baseline_cls_20260110_182220\best.pt
VAL summary: {'loss': 0.0409371746701072, 'acc': 0.9925, 'ap': 0.9997323957069084}
TEST summary: {'loss': 0.8746048719155787, 'acc': 0.8589743589743589, 'ap': 0.9496867496329111}
RUN_DIR = outputs\runs\medimg_baseline_cls_20260110_182220
BEST_CKPT = outputs\runs\medimg_baseline_cls_20260110_182220\best.pt


In [8]:
from pathlib import Path
from src.utils import save_json

# Save a pointer to the latest run for downstream notebooks
latest_path = Path("outputs") / "runs" / "_latest_run.json"
save_json(
    {"run_dir": result["run_dir"], "best_ckpt_path": result["best_ckpt_path"]},
    str(latest_path)
)

print("Wrote latest run pointer to:", latest_path)


Wrote latest run pointer to: outputs\runs\_latest_run.json
